In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# EStore_Test.ipynb

# Step 1: Import required modules
import os
import pandas as pd

# Import your transformer classes from src
from src.product_transformer import ProductTransformer
from src.cart_transformer import CartTransformer
from src.purchase_transformer import PurchaseTransformer
from src.discount_transformer import DiscountTransformer


In [3]:
from src.base_transformer import BaseDBTransformer
import inspect
print(inspect.getsource(BaseDBTransformer.transform))


    def transform(self, X=None):
        """Return current dataframe for pipeline usage."""
        print("Base Transformer transform invoked")
        if self.data is None:
            self.fit()
        return self.data.copy()



In [5]:
# Initialize
product_csv = "data/products.csv"
product_transformer = ProductTransformer(file_path=product_csv)

Base DB Object Transformer invoked with file path: data\products.csv


In [6]:
print(product_transformer.__class__.mro())       # confirm resolution order
print(product_transformer.transform)            # which function object is bound?
print(product_transformer.transform.__func__)   # unbound function, shows true owner


[<class 'src.product_transformer.ProductTransformer'>, <class 'src.base_transformer.BaseDBTransformer'>, <class 'object'>]
<bound method BaseDBTransformer.transform of <src.product_transformer.ProductTransformer object at 0x00000161DA7C3E00>>
<function BaseDBTransformer.transform at 0x00000161DA8D1BC0>


In [7]:
product_transformer.transform()  

Base Transformer transform invoked
Base Transformer fit invoked


AttributeError: module 'pandas' has no attribute 'read_DB'

In [ ]:



# Test: Read back
# print(product_transformer.read())

# Test: CRUD ops
print(product_transformer.create({C.pid: "FD6431", C.ptyp: "Fruit and Vegetables", C.sc: "Regular", C.wt: 10.57, C.area: 0.11, C.mrp: 143.36}))
print(product_transformer.read())


Base DB Object Transformer invoked with file path: data\products.csv


TypeError: BaseDBTransformer.transform() missing 1 required positional argument: 'X'

In [ ]:
# Initialize
discount_csv = "data/discounts.csv"
discount_transformer = DiscountTransformer(filepath=discount_csv)

# Test: Write sample data
sample_discounts = pd.DataFrame([
    {"discount_id": 1, "user_id": 101, "code": "NEW10", "percent": 10},
    {"discount_id": 2, "user_id": 102, "code": "SAVE20", "percent": 20}
])
discount_transformer.write_data(sample_discounts)

# Test: Read back
print(discount_transformer.read_data())

# Test: CRUD ops
print(discount_transformer.create({"discount_id": 3, "user_id": 103, "code": "FESTIVE30", "percent": 30}))
print(discount_transformer.read_all())


In [ ]:
# Initialize
purchase_csv = "data/purchases.csv"
purchase_transformer = PurchaseTransformer(filepath=purchase_csv)

# Test: Write sample data
sample_purchases = pd.DataFrame([
    {"purchase_id": 1, "user_id": 101, "product_id": 1, "quantity": 2, "price": 100, "discount": 0},
    {"purchase_id": 2, "user_id": 102, "product_id": 2, "quantity": 3, "price": 90, "discount": 10}
])
purchase_transformer.write_data(sample_purchases)

# Test: Read back
print(purchase_transformer.read_data())

# Test: CRUD ops
print(purchase_transformer.create({"purchase_id": 3, "user_id": 103, "product_id": 1, "quantity": 1, "price": 50, "discount": 5}))
print(purchase_transformer.read_all())


In [ ]:
# Initialize
cart_csv = "data/carts.csv"
cart_transformer = CartTransformer(filepath=cart_csv)

# Test: Write sample data
sample_cart = pd.DataFrame([
    {"cart_id": 1, "user_id": 101, "product_id": 1, "quantity": 2},
    {"cart_id": 2, "user_id": 102, "product_id": 2, "quantity": 3}
])
cart_transformer.write_data(sample_cart)

# Test: Read back
print(cart_transformer.read_data())

# Test: CRUD ops
print(cart_transformer.create({"cart_id": 3, "user_id": 103, "product_id": 1, "quantity": 1}))
print(cart_transformer.read_all())


In [ ]:
curl -X POST http://127.0.0.1:5000/products \
     -H "Content-Type: application/json" \
     -d '{"id": 1, "name": "Laptop", "price": 1200}'


In [ ]:
curl http://127.0.0.1:5000/products


In [ ]:
import time, random, string

def generate_user_id():
    timestamp = int(time.time())  # current Unix timestamp
    rand_part = ''.join(random.choices(string.ascii_letters + string.digits, k=6))
    return f"user_{timestamp}_{rand_part}"

print(generate_user_id())
